In [1]:
import numpy as np

# Load the saved data
X = np.load('notes.npy', allow_pickle=True)

In [2]:
print(X[:4])

[{'start': np.float64(1.0927083333333334), 'end': np.float64(1.1895833333333332), 'pitch': 71, 'velocity': 60}
 {'start': np.float64(1.2791666666666666), 'end': np.float64(1.496875), 'pitch': 55, 'velocity': 44}
 {'start': np.float64(1.4635416666666667), 'end': np.float64(1.6312499999999999), 'pitch': 59, 'velocity': 55}
 {'start': np.float64(1.6333333333333333), 'end': np.float64(1.753125), 'pitch': 62, 'velocity': 52}]


In [3]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

class MusicSequenceDataset(Dataset):
    def __init__(self, notes, sequence_length):
        self.notes = notes  # list of lists of dictionaries (list of sequences of notes)
        self.sequence_length = sequence_length  # length of the input sequence

    def __len__(self):
        return len(self.notes) - self.sequence_length  # Total number of sequences available

    def __getitem__(self, idx):
        # Get a sequence of notes (list of dictionaries)
        sequence = self.notes[idx:idx + self.sequence_length]
        target = self.notes[idx + self.sequence_length]

        # The sequence should now be a flat list of dictionaries, create the tensor for X
        X = torch.tensor([[note['start'], note['end'], note['pitch'], note['velocity']] for note in sequence], dtype=torch.float32)
        
        # The target note is the next note's features
        y = torch.tensor([target['start'], target['end'], target['pitch'], target['velocity']], dtype=torch.float32)
        
        return X, y

class MusicLSTMModel(nn.Module):
    def __init__(self, input_size=4, hidden_size=128, num_layers=2, output_size=4):
        super(MusicLSTMModel, self).__init__()
        
        # LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        
        # Fully connected layer to map the hidden state to the output
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        # Pass through LSTM layer
        lstm_out, (hn, cn) = self.lstm(x)
        
        # Only take the output of the last time step
        last_lstm_output = lstm_out[:, -1, :]
        
        # Pass the last output through a fully connected layer
        output = self.fc(last_lstm_output)
        
        return output
    
# Instantiate the dataset and DataLoader
sequence_length = 16
dataset = MusicSequenceDataset(X, sequence_length=sequence_length)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)
print(dataset[0])

# Assuming you have your DataLoader as `dataloader`
total_batches = len(dataloader)
print(f"Total number of batches: {total_batches}")


(tensor([[ 1.7865,  1.8281, 72.0000, 76.0000],
        [ 1.8031,  2.0000, 67.0000, 56.0000],
        [ 1.9833,  2.0979, 74.0000, 68.0000],
        [ 2.0375,  2.1063, 72.0000, 77.0000],
        [ 2.0979,  2.1823, 74.0000, 51.0000],
        [ 2.1719,  2.2802, 67.0000, 57.0000],
        [ 2.3281,  2.5094, 66.0000, 58.0000],
        [ 2.1490,  2.5198, 72.0000, 60.0000],
        [ 1.9833,  2.5229, 57.0000, 61.0000],
        [ 2.5229,  2.5896, 71.0000, 68.0000],
        [ 2.5906,  2.6740, 72.0000, 47.0000],
        [ 2.5583,  2.7635, 64.0000, 35.0000],
        [ 2.8875,  3.1135, 62.0000, 63.0000],
        [ 3.0802,  3.2729, 66.0000, 63.0000],
        [ 2.7125,  3.4198, 59.0000, 50.0000],
        [ 2.6792,  3.4479, 74.0000, 68.0000]]), tensor([ 3.4531,  3.6021, 71.0000, 66.0000]))
Total number of batches: 110003


In [4]:
import torch.optim as optim
from tqdm import tqdm
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize the model and move it to the correct device
model = MusicLSTMModel()
model = model.to(device)

epochs = 3
learning_rate = 0.001
criterion = nn.MSELoss()  # Mean Squared Error for regression tasks
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Example of using a DataLoader (ensure it's initialized correctly elsewhere)
# train_loader = DataLoader(...)

for epoch in range(epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    # Loop over batches in the DataLoader
    for batch_idx, (X_batch, y_batch) in tqdm(enumerate(dataloader), bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed} < {remaining}, {rate_fmt}]"):
        optimizer.zero_grad()  # Zero out previous gradients

        # Move batch data to device (GPU or CPU)
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        # Forward pass
        outputs = model(X_batch)

        # Compute loss
        loss = criterion(outputs, y_batch)  # Compare predicted outputs with actual targets

        # Backward pass and optimization
        loss.backward()  # Compute gradients
        optimizer.step()  # Update the model parameters

        # Accumulate the running loss
        running_loss += loss.item()

    # Print the loss for this epoch
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss / len(dataloader):.4f}")

torch.save(model.state_dict(), f"model_epoch_{epoch+1}.pth")

|          | 110003/? [07:15 < 00:00, 252.67it/s]


Epoch [1/3], Loss: 7642.0334


|          | 110003/? [07:08 < 00:00, 256.79it/s]


Epoch [2/3], Loss: 447.0842


|          | 110003/? [07:08 < 00:00, 256.73it/s]

Epoch [3/3], Loss: 424.5167
